# 01 – Innhenting av meningsmålinger (Wikipedia)

Denne notebooken henter tabeller fra Wikipedia og lagrer i SQLite.

In [ ]:
# Oppsett
import sys, os
sys.path.append(os.path.abspath("src"))
from src.data.wikipedia import fetch_wikipedia_polls
from src.db import get_conn, init_db, insert_poll
import pandas as pd


In [ ]:
# Hent tabell(er) fra Wikipedia
df = fetch_wikipedia_polls()
print(df.shape)
df.head(10)

In [ ]:
# Skriv til SQLite
conn = get_conn()
init_db(conn)
n = 0
for _, row in df.iterrows():
    meta = {
        'source': 'wikipedia',
        'pollster': row.get('pollster'),
        'client': row.get('client'),
        'fieldwork_start': row.get('fieldwork_start'),
        'fieldwork_end': row.get('fieldwork_end'),
        'publish_date': None,
        'sample_size': int(row['sample_size']) if pd.notnull(row.get('sample_size')) else None,
        'population': row.get('population'),
        'url': row.get('url')
    }
    results = {p: row.get(p) for p in ['Ap','H','FrP','SV','Sp','V','KrF','MDG','R','Andre'] if p in row.index}
    insert_poll(conn, meta, results)
    n += 1
print("Innsatte rader:", n)

In [ ]:
# Sjekk at data finnes
import pandas as pd
con = get_conn()
polls = pd.read_sql_query('SELECT * FROM polls ORDER BY fieldwork_end DESC LIMIT 5', con)
results = pd.read_sql_query('SELECT * FROM poll_results WHERE poll_id IN (SELECT id FROM polls ORDER BY fieldwork_end DESC LIMIT 5)', con)
polls, results.head(20)